## Case Studies Analysis

In [ ]:
## import python package
import os, sys
path_list = os.getcwd().split('/')
index = path_list.index('KGML-xDTD')
script_path = '/'.join(path_list[:(index+1)] + ['model_evaluation','scripts'])
sys.path.append(script_path)
import pandas as pd
import eval_utilities
from KGML_xDTD import KGML_xDTD

import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

In [ ]:
## define some Classes and functions

class Args:
    def __init__(self, use_gpu: bool = True, gpu: int = 0):
        self.use_gpu = use_gpu
        self.gpu = gpu

def my_draw_networkx_edge_labels(
    G,
    pos,
    edge_labels=None,
    label_pos=0.5,
    font_size=10,
    font_color="k",
    font_family="sans-serif",
    font_weight="normal",
    alpha=None,
    bbox=None,
    horizontalalignment="center",
    verticalalignment="center",
    ax=None,
    rotate=True,
    clip_on=True,
    rad=0
):
    import matplotlib.pyplot as plt
    import numpy as np

    if ax is None:
        ax = plt.gca()
    if edge_labels is None:
        labels = {(u, v): d for u, v, d in G.edges(data=True)}
    else:
        labels = edge_labels
    text_items = {}
    for (n1, n2), label in labels.items():
        (x1, y1) = pos[n1]
        (x2, y2) = pos[n2]
        (x, y) = (
            x1 * label_pos + x2 * (1.0 - label_pos),
            y1 * label_pos + y2 * (1.0 - label_pos),
        )
        pos_1 = ax.transData.transform(np.array(pos[n1]))
        pos_2 = ax.transData.transform(np.array(pos[n2]))
        linear_mid = 0.5*pos_1 + 0.5*pos_2
        d_pos = pos_2 - pos_1
        rotation_matrix = np.array([(0,1), (-1,0)])
        ctrl_1 = linear_mid + rad*rotation_matrix@d_pos
        ctrl_mid_1 = 0.5*pos_1 + 0.5*ctrl_1
        ctrl_mid_2 = 0.5*pos_2 + 0.5*ctrl_1
        bezier_mid = 0.5*ctrl_mid_1 + 0.5*ctrl_mid_2
        (x, y) = ax.transData.inverted().transform(bezier_mid)

        if rotate:
            # in degrees
            angle = np.arctan2(y2 - y1, x2 - x1) / (2.0 * np.pi) * 360
            # make label orientation "right-side-up"
            if angle > 90:
                angle -= 180
            if angle < -90:
                angle += 180
            # transform data coordinate angle to screen coordinate angle
            xy = np.array((x, y))
            trans_angle = ax.transData.transform_angles(
                np.array((angle,)), xy.reshape((1, 2))
            )[0]
        else:
            trans_angle = 0.0
        # use default box of white with white border
        if bbox is None:
            bbox = dict(boxstyle="round", ec=(1.0, 1.0, 1.0), fc=(1.0, 1.0, 1.0))
        if not isinstance(label, str):
            label = str(label)  # this makes "1" and 1 labeled the same

        t = ax.text(
            x,
            y,
            label,
            size=font_size,
            color=font_color,
            family=font_family,
            weight=font_weight,
            alpha=alpha,
            horizontalalignment=horizontalalignment,
            verticalalignment=verticalalignment,
            rotation=trans_angle,
            transform=ax.transData,
            bbox=bbox,
            zorder=1,
            clip_on=clip_on,
        )
        text_items[(n1, n2)] = t

    ax.tick_params(
        axis="both",
        which="both",
        bottom=False,
        left=False,
        labelbottom=False,
        labelleft=False,
    )

    return text_items    

def reorganize_df(edges):
    temp = dict()
    for index in range(len(edges)):
        source, target = edges.loc[index,0],edges.loc[index,1]
        if (source, target) not in temp:
            temp[(source, target)] = dict()
            temp[(source, target)]['edges'] = set([edges.loc[index,2].replace('biolink:','')])
        else:
            temp[(source, target)]['edges'].update(set([edges.loc[index,2].replace('biolink:','')]))

    return pd.DataFrame([(key[0],key[1],'/'.join(list(value['edges']))) for key, value in temp.items()])

def path_graph(path_res_dict, pair, title, tp_score, is_in_train_set, is_in_not_train_set, rad=0.07):
    edges = reorganize_df(path_res_dict[pair])
    g = nx.DiGraph()
    g.add_edges_from([(x[0],x[1], {'relation': x[2].replace('biolink:','')}) for x in edges.to_numpy()])
    labels= nx.get_edge_attributes(g,'relation')

    f = plt.figure(figsize=(18, 12));
    ax = f.add_subplot(111);
    pos = graphviz_layout(g, prog='dot', args='-Grankdir="LR"');
    nx.draw_networkx_nodes(g, pos, node_size=10, linewidths=0.5, alpha=0.9)
    nx.draw_networkx_edges(g, pos, connectionstyle=f'arc3,rad={rad}', width=0.5)
    nx.draw_networkx_labels(g, pos, font_size=12, font_color='blue');
    my_draw_networkx_edge_labels(g, pos, edge_labels=labels, rotate=True, font_size=6, font_weight='bold', rad = rad);
    plt.text(0.005, 0.06, f"Predicted Score: {tp_score}", size=10, color='black', fontweight='bold', transform=ax.transAxes);
    plt.text(0.005, 0.03, f"Is in train set: {is_in_train_set}", size=10, color='black', fontweight='bold', transform=ax.transAxes);
    plt.text(0.005, 0, f"Is in val/test set: {is_in_not_train_set}", size=10, color='black', fontweight='bold', transform=ax.transAxes);
    plt.text(0.005, 0.97, f"Drug ID in KG2c: {pair[0]} ({title.split(' - ')[0]})", size=10, color='black', fontweight='bold', transform=ax.transAxes);
    plt.text(0.005, 0.94, f"Disease ID in KG2c: {pair[1]} ({title.split(' - ')[1]})", size=10, color='black', fontweight='bold', transform=ax.transAxes);
    return [f,g]

In [ ]:
## set up general parameters
args = Args()

## set up data path and model path
data_path = '/'.join(path_list[:(index+1)] + ['model_evaluation','data'])
model_path = '/'.join(path_list[:(index+1)] + ['model_evaluation','models'])

## create a KGML-xDTD object (this step needs to take around 5 minutes because its need to load the required files (e.g. KG) and trained modules)
xdtd = KGML_xDTD(args, data_path, model_path)

## read training, validation and test datasets
train_rf_3class = pd.read_csv(os.path.join(data_path, 'train_pairs.txt'), sep='\t', header=0)
val_rf_3class = pd.read_csv(os.path.join(data_path, 'val_pairs.txt'), sep='\t', header=0)
test_rf_3class = pd.read_csv(os.path.join(data_path, 'test_pairs.txt'), sep='\t', header=0)

### Case Studies Analysis with Hemophilia B

In [ ]:
## predict top 100 potential drugs that could be used to treat a given disease
res = xdtd.predict_top_N_drugs(disease_name='Hemophilia B', N=100)
hemophilia_curie_id = xdtd._get_preferred_curie(name='Hemophilia B', curie_type='disease')
## take the top 10 drug-Hemophilia B pairs that includes at most 5 results in training set
tp_in_train_set = pd.DataFrame(res['drug_id'].isin(list(train_rf_3class.loc[(train_rf_3class['target'].isin([hemophilia_curie_id])) & (train_rf_3class['y']==1),'source'])))
tp_not_in_train_set_list = list(val_rf_3class.loc[(val_rf_3class['target'].isin([hemophilia_curie_id])) & (val_rf_3class['y']==1),'source']) + list(test_rf_3class.loc[(test_rf_3class['target'].isin([hemophilia_curie_id])) & (test_rf_3class['y']==1),'source'])
tp_not_in_train_set = pd.DataFrame(res['drug_id'].isin(tp_not_in_train_set_list))
tn_in_train_set = pd.DataFrame(res['drug_id'].isin(list(train_rf_3class.loc[(train_rf_3class['target'].isin([hemophilia_curie_id])) & (train_rf_3class['y']==0),'source'])))
tn_not_in_train_set_list = list(val_rf_3class.loc[(val_rf_3class['target'].isin([hemophilia_curie_id])) & (val_rf_3class['y']==0),'source']) + list(test_rf_3class.loc[(test_rf_3class['target'].isin([hemophilia_curie_id])) & (test_rf_3class['y']==0),'source'])
tn_not_in_train_set = pd.DataFrame(res['drug_id'].isin(tn_not_in_train_set_list))
random_pairs_in_train_set = pd.DataFrame(res['drug_id'].isin(list(train_rf_3class.loc[(train_rf_3class['target'].isin([hemophilia_curie_id])) & (train_rf_3class['y']==2),'source'])))
random_pairs_not_in_train_set_list = list(val_rf_3class.loc[(val_rf_3class['target'].isin([hemophilia_curie_id])) & (val_rf_3class['y']==2),'source']) + list(test_rf_3class.loc[(test_rf_3class['target'].isin([hemophilia_curie_id])) & (test_rf_3class['y']==2),'source'])
random_pairs_not_in_train_set = pd.DataFrame(res['drug_id'].isin(random_pairs_not_in_train_set_list))
res = pd.concat([res,tp_in_train_set,tp_not_in_train_set,tn_in_train_set,tn_not_in_train_set,random_pairs_in_train_set,random_pairs_not_in_train_set], axis=1)
res.columns = ['drug_id', 'drug_name', 'tn_score', 'tp_score', 'unknown_score', 'tp_in_train_set', 'tp_not_in_train_set', 'tn_in_train_set','tn_not_in_train_set','random_pairs_in_train_set','random_pairs_not_in_train_set']
temp = pd.concat([res.loc[res['tp_in_train_set'],:][:5],res.loc[~res['tp_in_train_set'],:]]).reset_index(drop=True)
hemophilia_b_top10 = temp.loc[:9,:].reset_index(drop=True)

In [ ]:
## predict top 10 KG-based MOA paths for each of top 10 drug-Hemophilia B pairs that includes at most 5 results in training set
path_res = dict()
for row in hemophilia_b_top10.to_numpy():
    drug_curie = row[0]
    disease_curie = hemophilia_curie_id
    temp = xdtd.predict_top_M_moa_paths(drug_curie=drug_curie, disease_curie=disease_curie, M=10)
    path_res[(drug_curie, disease_curie)] = temp

In [ ]:
## integrate 10 paths into a subgraph for each drug-disease pair
path_res_dict = dict()
for pair in path_res:
    path_segment_set = set()
    if path_res[pair]:
        for path in path_res[pair]:
            path_segment = path[0].split('->')
            temp = set([(path_segment[index],path_segment[index+2],path_segment[index+1]) for index in range(0,len(path_segment)-2,2)])
            path_segment_set.update(temp)
        path_res_dict[pair] = pd.DataFrame(path_segment_set)
    else:
        print(f'No KG-based paths with length up to 3 for pair {pair}')

for pair in path_res_dict:
    if path_res_dict[pair].shape[0] == 0:
        continue
    drug_name = eval_utilities.id_to_name(pair[0]).capitalize()
    disease_name = eval_utilities.id_to_name(pair[1]).capitalize()
    title = f"{drug_name} - {disease_name}"
    tp_score = round(hemophilia_b_top10.loc[hemophilia_b_top10['drug_id']==pair[0],'tp_score'].item(),6)
    is_in_train_set = str(hemophilia_b_top10.loc[hemophilia_b_top10['drug_id']==pair[0],'tp_in_train_set'].to_numpy().item())
    is_in_not_train_set = str(hemophilia_b_top10.loc[hemophilia_b_top10['drug_id']==pair[0],'tp_not_in_train_set'].to_numpy().item())
    fig, g = path_graph(path_res_dict, pair, title, tp_score, is_in_train_set, is_in_not_train_set)

### Case Studies Analysis with Huntington disease

In [ ]:
## predict top 100 potential drugs that could be used to treat a given disease
res = xdtd.predict_top_N_drugs(disease_name='Huntington disease', N=100)
huntington_curie_id = xdtd._get_preferred_curie(name='Huntington disease', curie_type='disease')
## take the top 10 drug-Huntington disease pairs that includes at most 5 results in training set
tp_in_train_set = pd.DataFrame(res['drug_id'].isin(list(train_rf_3class.loc[(train_rf_3class['target'].isin([huntington_curie_id])) & (train_rf_3class['y']==1),'source'])))
tp_not_in_train_set_list = list(val_rf_3class.loc[(val_rf_3class['target'].isin([huntington_curie_id])) & (val_rf_3class['y']==1),'source']) + list(test_rf_3class.loc[(test_rf_3class['target'].isin([huntington_curie_id])) & (test_rf_3class['y']==1),'source'])
tp_not_in_train_set = pd.DataFrame(res['drug_id'].isin(tp_not_in_train_set_list))
tn_in_train_set = pd.DataFrame(res['drug_id'].isin(list(train_rf_3class.loc[(train_rf_3class['target'].isin([huntington_curie_id])) & (train_rf_3class['y']==0),'source'])))
tn_not_in_train_set_list = list(val_rf_3class.loc[(val_rf_3class['target'].isin([huntington_curie_id])) & (val_rf_3class['y']==0),'source']) + list(test_rf_3class.loc[(test_rf_3class['target'].isin([huntington_curie_id])) & (test_rf_3class['y']==0),'source'])
tn_not_in_train_set = pd.DataFrame(res['drug_id'].isin(tn_not_in_train_set_list))
random_pairs_in_train_set = pd.DataFrame(res['drug_id'].isin(list(train_rf_3class.loc[(train_rf_3class['target'].isin([huntington_curie_id])) & (train_rf_3class['y']==2),'source'])))
random_pairs_not_in_train_set_list = list(val_rf_3class.loc[(val_rf_3class['target'].isin([huntington_curie_id])) & (val_rf_3class['y']==2),'source']) + list(test_rf_3class.loc[(test_rf_3class['target'].isin([huntington_curie_id])) & (test_rf_3class['y']==2),'source'])
random_pairs_not_in_train_set = pd.DataFrame(res['drug_id'].isin(random_pairs_not_in_train_set_list))
res = pd.concat([res,tp_in_train_set,tp_not_in_train_set,tn_in_train_set,tn_not_in_train_set,random_pairs_in_train_set,random_pairs_not_in_train_set], axis=1)
res.columns = ['drug_id', 'drug_name', 'tn_score', 'tp_score', 'unknown_score', 'tp_in_train_set', 'tp_not_in_train_set', 'tn_in_train_set','tn_not_in_train_set','random_pairs_in_train_set','random_pairs_not_in_train_set']
temp = pd.concat([res.loc[res['tp_in_train_set'],:][:5],res.loc[~res['tp_in_train_set'],:]]).reset_index(drop=True)
## remove chemotherapy drugs
chemotherapy_medication = ['DOCETAXEL', 'CYCLOPHOSPHAMIDE', 'CISPLATIN', 'IMATINIB', 'METHOTREXATE']
temp = temp.loc[~temp['drug_name'].isin(chemotherapy_medication),:].reset_index(drop=True)
huntington_top10 = temp.loc[:9,:].reset_index(drop=True)


In [ ]:
## predict top 10 KG-based MOA paths for each of top 10 drug-Huntington disease pairs that includes at most 5 results in training set
path_res = dict()
for row in huntington_top10.to_numpy():
    drug_curie = row[0]
    disease_curie = huntington_curie_id
    temp = xdtd.predict_top_M_moa_paths(drug_curie=drug_curie, disease_curie=disease_curie, M=10)
    path_res[(drug_curie, disease_curie)] = temp

In [ ]:
## integrate 10 paths into a subgraph for each drug-disease pair
path_res_dict = dict()
for pair in path_res:
    path_segment_set = set()
    if path_res[pair]:
        for path in path_res[pair]:
            path_segment = path[0].split('->')
            temp = set([(path_segment[index],path_segment[index+2],path_segment[index+1]) for index in range(0,len(path_segment)-2,2)])
            path_segment_set.update(temp)
        path_res_dict[pair] = pd.DataFrame(path_segment_set)
    else:
        print(f'No KG-based paths with length up to 3 for pair {pair}')

for pair in path_res_dict:
    if path_res_dict[pair].shape[0] == 0:
        continue
    drug_name = eval_utilities.id_to_name(pair[0]).capitalize()
    disease_name = eval_utilities.id_to_name(pair[1]).capitalize()
    title = f"{drug_name} - {disease_name}"
    tp_score = round(hemophilia_b_top10.loc[hemophilia_b_top10['drug_id']==pair[0],'tp_score'].item(),6)
    is_in_train_set = str(hemophilia_b_top10.loc[hemophilia_b_top10['drug_id']==pair[0],'tp_in_train_set'].to_numpy().item())
    is_in_not_train_set = str(hemophilia_b_top10.loc[hemophilia_b_top10['drug_id']==pair[0],'tp_not_in_train_set'].to_numpy().item())
    fig, g = path_graph(path_res_dict, pair, title, tp_score, is_in_train_set, is_in_not_train_set)